In [1]:
%matplotlib inline
from collections import Counter
import douwelib as dl
import os, glob, statistics, numpy as np, matplotlib.pyplot as plt, pandas as pd
from matplotlib import gridspec
from matplotlib.backends.backend_pdf import PdfPages
from collections import defaultdict
import matplotlib.patches as mpatches
import itertools
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, GlyphRenderer, Range1d
from bokeh.layouts import gridplot
import math
from bokeh.io import export_png


In [2]:
sequence_file = 'C:/Users/Douwe/Documents/Python/Sequence_files/COG_LAB008_20170217_FNFAF05255_MN17942_sequencing_run_RCA_Pool_2_1D_16122_ch90_read1116_strand.done_fastq'

In [3]:
size = 4
overlap = 1
letters = 'ACTG'

In [ ]:
#Automatic plot 
X = np.array(dl.get_list_fastq(sequence_file, size, overlap, letters)[0])
Y_h = dl.get_list_fastq(sequence_file, size, overlap, letters)[1]
Y_m = dl.get_list_fastq(sequence_file, size, overlap, letters)[2]
Y_l = np.array(dl.get_list_fastq(sequence_file, size, overlap, letters)[3])
dl.plot_bases_high_medium_low(X, Y_h, Y_m, Y_l, 1000, 10)

In [ ]:
#What is inside automatic function
data = dl.parse_fasta_file_error(sequence_file)          #reads sequence_file -> makes dictonairy with key = id and value = bases and scores
#print(data)
                                                         
id_ = liast(data.keys())[0]                               #makes name id_ from key
error_rate = dl.convert_qualityscore (data[id_]['score']) #converts quality score from symboles to values out of dict

nucleotide_string = (data[id_]['sequence'])
nucleotides_list = [c for c in (data[id_]['sequence'])]  #individualize nucleotides into list out of dict
X = np.array(nucleotides_list)                           #array for nucleotides and error rate
Y = np.array(error_rate)                                 

In [ ]:
scoremean_dict = dl.dict_scoremean_bases(nucleotide_string, error_rate, size, overlap)

#Now for creating plot with High, Medium and Low
plot_data_h = dl.dict_of_allbases_vs_High_Medium_Low(4, 'High', scoremean_dict, letters)
plot_data_m = dl.dict_of_allbases_vs_High_Medium_Low(4, 'Medium', scoremean_dict, letters)
plot_data_l = dl.dict_of_allbases_vs_High_Medium_Low(4, 'Low', scoremean_dict, letters)

X = np.array(list(plot_data_h.keys()))
Y_h = np.array(list(plot_data_h.values()))
Y_m = np.array(list(plot_data_m.values()))
Y_l = np.array(list(plot_data_l.values()))

plot_data_h.values()

In [ ]:
#For all the files

In [4]:
os.chdir('C:/Users/Douwe/Documents/Python/Sequence_files')
all_y_h = [] 
all_y_m = []
all_y_l = []
size = 5
overlap = 2
letters = 'ACTG'

for file in glob.iglob('*.done_fastq'):
    X = dl.get_list_fastq(file, size, overlap, letters)[0]
    y_h = (dl.get_list_fastq(file, size, overlap, letters)[1])
    y_m = (dl.get_list_fastq(file, size, overlap, letters)[2])
    y_l = (dl.get_list_fastq(file, size, overlap, letters)[3])
    all_y_h.append(y_h)
    all_y_m.append(y_m)
    all_y_l.append(y_l)
    
nc_all_y_h = [[i[n] for i in all_y_h] for n in range(len(all_y_h[0]))]
c_all_y_h = list(sum(x) for x in nc_all_y_h)
nc_all_y_m = [[i[n] for i in all_y_m] for n in range(len(all_y_m[0]))]
c_all_y_m = list(sum(x) for x in nc_all_y_m)
nc_all_y_l = [[i[n] for i in all_y_l] for n in range(len(all_y_l[0]))]
c_all_y_l = list(sum(x) for x in nc_all_y_l)

#dl.plot_bases_high_medium_low(X, c_all_y_h, c_all_y_m, c_all_y_l, 1000, 500)

#get dict with many high error rates and thus low quality
all_combinations = []
for i in itertools.product('ACTG', repeat = size):
    size_ = '{}'*size
    all_combinations.append(size_.format(*i))
    
all_high_scores = dict(zip(all_combinations, c_all_y_h))
d = dict((k, v) for k, v in all_high_scores.items() if v >= 60)


In [5]:
y_nan1 = dl.zero_to_nan(c_all_y_h)
y_nan2 = dl.zero_to_nan(c_all_y_m)
y_nan3 = dl.zero_to_nan(c_all_y_l)

In [ ]:
path = 'C:/Users/Douwe/Documents/Python/Sequence_files'
dl.get_list_hml_scores_fastq(path, size, overlap, letters)[1]

In [ ]:
#Plot in Bokeh
bases = 'ACTG'

#percentage
sum_scores = [x + y + z for x, y, z in zip(y_nan1, y_nan2, y_nan3)]
per_h_of_all = [x / y * 100 for x, y in zip(y_nan1, sum_scores)]
per_m_of_all = [x / y * 100 for x, y in zip(y_nan2, sum_scores)]
per_l_of_all = [x / y * 100 for x, y in zip(y_nan3, sum_scores)]

hover = HoverTool(tooltips=[("Bases", "@x"), ("High Scores", "@y""%")], mode='vline')

#plotting
p = figure(x_range=bases, y_axis_label='How many found bases in category%', tools=[hover, 'xwheel_zoom', 'xpan'])
p.title.text = "High, Medium, Low quality score per sequence"
p.xaxis.ticker = []
p.y_range = Range1d(0, 100)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#data
p.circle(x=bases, y=per_h_of_all, size=5, alpha=1, muted_alpha=0.4, color='red', muted_color='red', legend='High scores')
p.circle(x=bases, y=per_m_of_all, size=5, alpha=1, muted_alpha=0.4, color='blue', muted_color='blue', legend='Medium scores')
p.circle(x=bases, y=per_l_of_all, size=5, alpha=1, muted_alpha=0.4, color='purple', muted_color='purple', legend='Low scores')

p.legend.location = "top_left"
p.legend.click_policy="mute"

# output to static HTML file
#output_file("scatterplot.html", title="High, Medium, Low quality score per sequence")

show(p)

In [6]:
dl.plot_bases_hml_in_bokeh('D', X, y_nan1, y_nan2, y_nan3)

Figure(id='0c2db64b-a1e7-44b3-8122-711f7c2dca58', ...)

In [ ]:
#making dataframe for clustering
all_combinations = []
for i in itertools.product('ACTG', repeat = size):
    size_ = '{}'*size
    all_combinations.append(size_.format(*i))
    
all_high_scores = dict(zip(all_combinations, y_nan1))
#d_h = dict((k, v) for k, v in all_high_scores.items() if v >= 60)

all_combinations = []
for i in itertools.product('ACTG', repeat = size):
    size_ = '{}'*size
    all_combinations.append(size_.format(*i))
    
all_medium_scores = dict(zip(all_combinations, y_nan2))
#d_m = dict((k, v) for k, v in all_medium_scores.items() if v >= 60)

all_combinations = []
for i in itertools.product('ACTG', repeat = size):
    size_ = '{}'*size
    all_combinations.append(size_.format(*i))
    
all_low_scores = dict(zip(all_combinations, y_nan3))
#d_l = dict((k, v) for k, v in all_low_scores.items() if v >= 60)

all_high_scores['Scores'] = 'High'
all_medium_scores['Scores'] = 'Medium'
all_low_scores['Scores'] = 'Low'

df_ = pd.DataFrame.from_dict([all_high_scores, all_medium_scores, all_low_scores])
df_ = df_.set_index('Scores')
df_.columns.name = 'Sequence'
df_ = df_.fillna(value=0)

sum_scores = df_.sum(axis=0)
df_[0:1] = (df_[0:1]/sum_scores)*100
df_[1:2] = (df_[1:2]/sum_scores)*100
df_[2:3] = (df_[2:3]/sum_scores)*100

In [ ]:
#plotting with clustering in bokeh
from sklearn.cluster import KMeans

x_plot = np.column_stack(([df_.iloc[2], df_.iloc[0]]))
kmeans= KMeans(n_clusters=4)
kmeans.fit(x_plot)
y_kmeans = kmeans.predict(x_plot)
colors = np.array([x for x in ('#00f', '#f00', '#0ff', '#ff0')])

sequence = list(df_.columns)
TOOLS='hover, reset, pan, wheel_zoom'

p = figure(x_range=[0, 100], y_range=[0, 100], x_axis_label='% in low score', y_axis_label='% in high score', 
           title='Clustering of Qualityscore', tools=TOOLS)
p.circle(x_plot[:, 0], x_plot[:, 1], color=colors[y_kmeans].tolist(), fill_alpha=0.2)
centers = kmeans.cluster_centers_
p.circle(centers[:, 0], centers[:, 1], color='black', size=10, alpha=0.4);

p.select_one(HoverTool).tooltips = [
     ('HighScore', '$x%'),
     ('LowScore', '$y%')
]

show(p)

In [ ]:
all_combinations = dl.list_with_all_combinations(letters, size)

per_all_h_scores = dict(zip(all_combinations, per_h_of_all))

In [ ]:
per_all_h_scores 
d = dict((k, v) for k, v in per_all_h_scores.items() if v >= 20)
print(d)

In [ ]:
#scores as values
os.chdir('C:/Users/Douwe/Documents/Python/Sequence_files')
dict_x = dict.fromkeys(dl.list_with_all_combinations(letters, size))

for file in glob.iglob('*.done_fastq'):
    data = dl.parse_fasta_file_error(file)
    id_ = list(data.keys())[0]
    error_rate = (dl.convert_qualityscore(data[id_]['score']))
    nucleotide_string = ((data[id_]['sequence']))
    n_split = dl.split_overlap(nucleotide_string, size, overlap)
    listed_nucleotides = []
    for l in n_split:
        if len(l) >= size:
            listed_nucleotides.append(l)
        else:
            #print('string without {} bases were skipped'.format(size))
            continue
    error_rate_split = dl.mean_calc_list(list(dl.split_overlap(error_rate, size, overlap)), size)
    dict_z = dict(zip(listed_nucleotides, error_rate_split))
    dl.dict_values_to_other_dict(dict_x, dict_z)

for k, v in dict_x.items():
    if v == None:
        continue
    else:
        dict_x[k] = statistics.mean(v)

In [ ]:
path = 'C:/Users/Douwe/Documents/Python/Sequence_files'

dict_x = dl.get_dict_mean_of_seq_fastq(path, size, overlap, letters)

In [ ]:
from sklearn.cluster import KMeans

keys = []
values = []
for key, value in dict_x.items():
    keys.append(key)
    values.append(value)

x_plot = np.column_stack((keys, values))

#Plot in Bokeh
ylim = len(values)
figsize = 500

hover = HoverTool(tooltips=[("Bases", "@x"), ("Qualityscore", "@y")], mode='vline')

#plotting
p = figure(x_range=keys, y_axis_label='QaulityScore', tools=[hover, 'xwheel_zoom', 'xpan'])
p.title.text = "Quality Score per base combinations"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#data
p.circle(x = keys, y= values, size=5, alpha=1, color='red', legend='QualityScore')

p.legend.location = "top_left"

show(p)

In [ ]:
#for documentation. want 6 plots. 3 on top with size differences and overlap. and 3 on bottum with same parameters but with
#with last plot. so 4:1, 5:2, 6:3 high/medium/low and 4:1, 5:2, 6:3 normal scores
#Put in dl.get_list_hml_scores_fastq(path, size, overlap, letters)

path = 'C:/Users/Douwe/Documents/Python/Sequence_files'
letters = 'ACTG'

#data
x1 = dl.get_list_hml_scores_fastq(path, 4, 1, letters)[0]
x2 = dl.get_list_hml_scores_fastq(path, 5, 2, letters)[0]
x3 = dl.get_list_hml_scores_fastq(path, 6, 3, letters)[0]
y_high1 = dl.get_list_hml_scores_fastq(path, 4, 1, letters)[1]
y_high2 = dl.get_list_hml_scores_fastq(path, 5, 2, letters)[1]
y_high3 = dl.get_list_hml_scores_fastq(path, 6, 3, letters)[1]
y_medium1 = dl.get_list_hml_scores_fastq(path, 4, 1, letters)[2]
y_medium2 = dl.get_list_hml_scores_fastq(path, 5, 2, letters)[2]
y_medium3 = dl.get_list_hml_scores_fastq(path, 6, 3, letters)[2]
y_low1 = dl.get_list_hml_scores_fastq(path, 4, 1, letters)[3]
y_low2 = dl.get_list_hml_scores_fastq(path, 5, 2, letters)[3]
y_low3 = dl.get_list_hml_scores_fastq(path, 6, 3, letters)[3]

#actual plot
A = dl.plot_raw_values_bokeh('A', dl.get_dict_mean_of_seq_fastq(path, 4, 1, letters))
B = dl.plot_raw_values_bokeh('B', dl.get_dict_mean_of_seq_fastq(path, 5, 2, letters))
C = dl.plot_raw_values_bokeh('C', dl.get_dict_mean_of_seq_fastq(path, 6, 2, letters))
D = dl.plot_bases_hml_in_bokeh('D', x1, y_high1, y_medium1, y_low1)
E = dl.plot_bases_hml_in_bokeh('E', x2, y_high2, y_medium2, y_low2)
F = dl.plot_bases_hml_in_bokeh('F', x3, y_high3, y_medium3, y_low3)

In [ ]:
grid = gridplot([[A, B, C], [D, E, F]])

export_png(grid, filename="plot.png")

#output_file("gridplot.png")
#output_file("gridplot.html")
#output to static HTML file
#output_file("gridplot.html", title="all data")

In [ ]:
show(grid)

In [ ]:
q1 = dl.get_dict_mean_of_seq_fastq(path, 4, 1, letters)
q2 = dl.get_dict_mean_of_seq_fastq(path, 5, 2, letters)
q3 = dl.get_dict_mean_of_seq_fastq(path, 6, 2, letters)

In [ ]:
sorted(q3.items(), key=lambda x:x[1])

In [ ]:

all_combinations = dl.list_with_all_combinations(letters, size)

per_all_h_scores = dict(zip(all_combinations, per_h_of_all))

per_all_h_scores 
d = dict((k, v) for k, v in per_all_h_scores.items() if v >= 20)
print(d)

In [ ]:
size = 6

sum_scores = [x + y + z for x, y, z in zip(y_high3, y_medium3, y_low3)]
per_h_of_all = [x / y * 100 for x, y in zip(y_high3, sum_scores)]
all_combinations = dl.list_with_all_combinations(letters, size)
z = dict(zip(all_combinations, per_h_of_all))


In [ ]:
sorted(z.items(), key=lambda x:x[1])

In [ ]:
t = list(z.values())
#q4 = [0 if t is NaN else t for t in z.values()]

q4 = [0 if math.isnan(t) else t for t in z.values()]

In [ ]:
t2 = list(z.keys())

In [ ]:
dict_z = dict(zip(t2, q4))

In [ ]:
dict_z

In [ ]:
sorted(dict_z.items(), key=lambda x:x[1])[4091:4097]

In [ ]:
len(dict_z)